In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

######HyperParameter
train_step = 20
batch_size = 100
fc_hidden1 = 120
fc_hidden2 = 84
lr = 0.1

#construct graph######################
x_batch = tf.placeholder(tf.float32,shape=[None,28,28,1],name="x_batch")
y_batch = tf.placeholder(tf.float32,shape=[None,10],name="y_batch")

####conv layer###################
def add_conv_layer(input,in_size,out_size,k_size,k_strides,activation_func=None):
    
#     W = tf.Variable(tf.truncated_normal(shape= [k_size, k_size, in_channel, out_channel], stddev = 0.1))
#     b = tf.Variable(tf.constant(0.1, shape=[out_channel]))
    
    W = tf.Variable(tf.truncated_normal(shape=[k_size,k_size,in_size,out_size], stddev = 0.1),name="W_conv")
    b = tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[out_size]),name="b_conv")
    
    
#     W = tf.Variable(tf.truncated_normal(shape=[k_size,k_size,in_size,out_size]),name="W_conv")
#     b = tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[out_size]),name="b_conv")
    
    xW_plus_b = tf.nn.conv2d(input,W,strides=[1,k_strides,k_strides,1],padding="SAME") + b
    if activation_func is None:
        return  xW_plus_b
    else:
        return activation_func(xW_plus_b)

#####pool layer##################
def add_pool_layer(input,k_size,k_strides,activation_func=None):
    return tf.nn.max_pool(input,ksize=[1,k_size,k_size,1],strides=[1,k_strides,k_strides,1],padding="SAME")

#####fully connectly layer##################
def add_fc_layer(input,in_size,out_size,activation_func=None):
    
#     W = tf.Variable(tf.truncated_normal(shape = [in_size, out_size], stddev = 0.1))
#     b = tf.Variable(tf.constant(0.1, shape = [out_size]))
    
    W = tf.Variable(tf.truncated_normal(shape=[in_size,out_size], stddev = 0.1),name="W_fc")
    b = tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[out_size]),name="b_fc")
    
    
#     W = tf.Variable(tf.truncated_normal(shape=[in_size,out_size]),name="W_fc")
#     b = tf.Variable(tf.constant(0.1,dtype=tf.float32,shape=[out_size]),name="b_fc")
    
    
    xW_plus_b = tf.matmul(input,W) + b
    if activation_func is None:
        return xW_plus_b
    else:
        return activation_func(xW_plus_b)

#####conv1_op##################
conv1 = add_conv_layer(x_batch,1,32,3,1,activation_func=tf.nn.relu)
#conv1[None,28,28,32]
#####pool1_op##################
pool1 = add_pool_layer(conv1,2,2,activation_func=None)
#pool1[None,14,14,32]
######conv2_op#################
conv2 = add_conv_layer(pool1,32,64,3,1,activation_func=tf.nn.relu)
#conv2[None,14,14,64]
######pool2_op#################
pool2 = add_pool_layer(conv2,2,2)
#pool2[None,7,7,64]
######flatten#################
flatten = tf.reshape(pool2,shape=[-1,7*7*64],name="input_flatten")
######fully1 connectly_op################
fc1 = add_fc_layer(flatten,7*7*64,fc_hidden1,tf.nn.relu)
######fully2 connectly_op################
fc2 = add_fc_layer(fc1,fc_hidden1,fc_hidden2,tf.nn.relu)
######output_op################
output = add_fc_layer(fc2,fc_hidden2,10)
#######accuracy###################
pred = tf.nn.softmax(output)
equal_data = tf.equal(tf.argmax(pred,axis=1),tf.argmax(y_batch,axis=1))
accuracy = tf.reduce_mean(tf.cast(equal_data,dtype=tf.float32))
######loss_op################
loss = tf.losses.softmax_cross_entropy(y_batch,output)
######train_op################
train_op = tf.train.GradientDescentOptimizer(lr).minimize(loss)
######init_op################
init_op = tf.global_variables_initializer()


mnist_base_path="/Users/zhouwencheng/Desktop/Grass/data/picture/mnist"
# 载入数据集
mnist = input_data.read_data_sets(mnist_base_path, one_hot=True)

Extracting ./data/MNIST_data/train-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./data/MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./data/MNIST_data/t10k-labels-idx1-ubyte.gz


In [8]:
######executer graph################
with tf.Session() as sess:
    sess.run(init_op)
    for i in range(train_step):
        total_batch = int(mnist.train.num_examples // batch_size)
        for one_batch in range(total_batch):
            x_train,y_train=mnist.train.next_batch(batch_size)
            x_flatten_train = x_train.reshape([batch_size,28,28,1])
            sess.run(train_op,feed_dict={x_batch:x_flatten_train,y_batch:y_train})

        x_test,y_test = mnist.test.next_batch(10000)
        x_flatten_test = x_test.reshape([10000,28,28,1])
        acc,output_value = sess.run([accuracy,loss],feed_dict={x_batch:x_flatten_test,y_batch:y_test})
        print("step: ",i,"loss: ",output_value,"accuracy: ",acc)



step:  0 loss:  0.08737619 accuracy:  0.9718
step:  1 loss:  0.072331876 accuracy:  0.9771
step:  2 loss:  0.053904884 accuracy:  0.9828
step:  3 loss:  0.0432497 accuracy:  0.9858
step:  4 loss:  0.04030357 accuracy:  0.9855
step:  5 loss:  0.03683272 accuracy:  0.9869
step:  6 loss:  0.036150172 accuracy:  0.9876
step:  7 loss:  0.037228893 accuracy:  0.9874
step:  8 loss:  0.038176075 accuracy:  0.9876
step:  9 loss:  0.033081263 accuracy:  0.9897
step:  10 loss:  0.03585591 accuracy:  0.9882
step:  11 loss:  0.040684145 accuracy:  0.988
step:  12 loss:  0.03651812 accuracy:  0.9889
step:  13 loss:  0.03705407 accuracy:  0.9893
step:  14 loss:  0.040960394 accuracy:  0.9885
step:  15 loss:  0.036979143 accuracy:  0.9897
step:  16 loss:  0.036929823 accuracy:  0.9893
step:  17 loss:  0.038664453 accuracy:  0.9892
step:  18 loss:  0.048693977 accuracy:  0.9876
step:  19 loss:  0.03860829 accuracy:  0.9905
